In [1]:
N = ord("R") % 5 + 1;
print(N);

3


In [3]:
%pip install pandas opencv-python openpyxl

In [10]:
import pandas as pd
import cv2

class ImageProcessor:
    def __init__(self, variant_data):
        self.variant_data = variant_data
        self.colors = {
            'red': (0, 0, 255),
            'blue': (255, 0, 0),
            'cian': (0, 0, 255),
            'magenta': (255, 0, 255),
            'yellow': (0, 255, 255)
        }

    def convert_color(self, color_text):
        return self.colors.get(color_text, (0, 0, 0))

    def process_image(self, image_path):
        img = cv2.imread(image_path)
        if img is None:
            print(f"Failed to read the image {image_path}")
            return None

        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        glasses_color_bgr = self.convert_color(self.variant_data['glasses color'].values[0])
        image_size = tuple(map(int, self.variant_data['image size'].values[0].split('x')))
        #line_width = int(self.variant_data['line width'].values[0])
        line_width = 2;

        for (x, y, w, h) in faces:
            face_roi = img[y:y+h, x:x+w]
            face_gray = gray[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(face_gray)
            eye_centers = []

            for (ex, ey, ew, eh) in eyes:
                glass_x = int(ex + 0.5 * ew)
                glass_y = int(ey + 0.5 * eh)
                glass_radius = int(0.5 * max(ew, eh))

                cv2.circle(face_roi, (glass_x, glass_y), glass_radius, glasses_color_bgr, line_width)
                eye_centers.append((x + glass_x, y + glass_y))

            eye_centers = sorted(eye_centers, key=lambda x: x[0])
            
            if len(eye_centers) == 2:
                cv2.line(img, (eye_centers[0][0] + glass_radius, eye_centers[0][1]), (eye_centers[1][0] - glass_radius, eye_centers[1][1]), glasses_color_bgr, line_width)
                cv2.line(img, (eye_centers[0][0] - glass_radius, eye_centers[0][1]), (eye_centers[0][0] - 50, eye_centers[0][1]), glasses_color_bgr, line_width)
                cv2.line(img, (eye_centers[1][0] + glass_radius, eye_centers[1][1]), (eye_centers[1][0] + 50, eye_centers[1][1]), glasses_color_bgr, line_width)

            result_img = cv2.resize(img, image_size)
            result_image_path = f'{image_path[:-4]}_glasses.jpg'
            cv2.imwrite(result_image_path, result_img)

            return result_image_path


df = pd.read_excel('lab6.xlsx')
variant_data = df[df['N'] == 3]
file_path = "./Images/" + variant_data['file name'].values[0]
processor = ImageProcessor(variant_data)
result_image_path = processor.process_image(file_path)
print("Image saved.")

Image saved.
